# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成計數編碼以及搭配邏輯斯迴歸的預測 (In[4], Out[4], In[5], Out[5]) 
- 仿造範例, 完成雜湊編碼, 以及計數編碼+雜湊編碼 搭配邏輯斯迴歸的預測 (In[6], Out[6], In[7], Out[7]) 
- 試著回答上述執行結果的觀察

# 作業1
* 參考範例，將鐵達尼的艙位代碼( 'Cabin' )欄位使用特徵雜湊 / 標籤編碼 / 計數編碼三種轉換後，  
與其他類別型欄位一起預估生存機率

In [134]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

data_path = 'data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

# 資料的合併與取出
train_Y = df_train['Survived']                      # 取出目標值
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])                  # 合併 train、test
df.head()

# 忽略警告訊息
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [135]:
#只取類別值 (object) 型欄位, 存於 object_features 中
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':                           # 篩選出類別型欄位
        object_features.append(feature)             # 將此欄位名稱加入列表
print(f'{len(object_features)} Object Features : {object_features}\n')

# 只留類別型欄位
df = df[object_features]                            # 只留下類別型欄位
df = df.fillna('None')                              # 填補缺失值為 'None'
train_num = train_Y.shape[0]                        # 得到 train 資料數
df.head()

5 Object Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']



,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S


# 作業2
* 承上題，三者比較效果何者最好?

In [136]:
# 對照組 : 標籤編碼 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in df.columns:
    df_temp[c] = LabelEncoder().fit_transform(df[c])        # 進行 Label Encoding
train_X = df_temp[:train_num]

estimator = LogisticRegression()                            # 設置使用羅吉斯回歸
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())    # 計算分數
df_temp.head()

0.7800138095537004


,Name,Sex,Ticket,Cabin,Embarked
0,155,1,720,185,3
1,286,0,816,106,0
2,523,0,914,185,3
3,422,0,65,70,3
4,22,1,649,185,3


In [137]:
# 每欄位資料類別數量
df.nunique()

Name        1307
Sex            2
Ticket       929
Cabin        187
Embarked       4
dtype: int64

In [138]:
# 加上 'Cabin' 欄位的計數編碼
Cabin_df = df.groupby(['Cabin']).count()['Name'].reset_index()          # 進行['Cabin']資料分類，並計算每一類別數量 // 取出其中一欄(每欄都一樣) // reset_index 是為了加上Ticket原欄位
Cabin_df = Cabin_df.rename( columns={'Name' : 'Cabin_count'} )          # 因分類後，名字會不正確，重新命名
df = pd.merge(df, Cabin_df, on=['Cabin'], how='left' )                  # 用['Cabin'] 合併[原資料]，與[分類計數資料]
df.head()

,Name,Sex,Ticket,Cabin,Embarked,Cabin_count
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S,1014
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C,2
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S,1014
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S,2
4,"Allen, Mr. William Henry",male,373450,None,S,1014


In [139]:
# 'Cabin'計數編碼 + 邏輯斯迴歸 
# df_temp = pd.DataFrame()        
le = LabelEncoder()

for col in object_features:                     # [col]只取[object_features]中的資料，不包含['Cabin_count']
    df_temp[col] = le.fit_transform(df[col])    # 將其他特徵用 Label Encoding 處理 
df_temp['Cabin_count'] = df['Cabin_count']      # 加入['Cabin_count']資料        
train_X = df_temp[:train_num]                   # 取出 train 資料

estimator = LogisticRegression()
score = cross_val_score(estimator, train_X, train_Y, cv=5).mean()   # 計算分數
print('score : ', score)
df_temp.head()

score :  0.7822547234950725


,Name,Sex,Ticket,Cabin,Embarked,Cabin_count
0,155,1,720,185,3,1014
1,286,0,816,106,0,2
2,523,0,914,185,3,1014
3,422,0,65,70,3,2
4,22,1,649,185,3,1014


In [140]:
# 'Cabin'特徵雜湊 + 邏輯斯迴歸
df_temp = pd.DataFrame()

for col in object_features:                                                 # [col]只取[object_features]中的資料，不包含['Cabin_count']
    df_temp[col] = le.fit_transform(df[col])                                # 將其他特徵用 Label Encoding 處理 
df_temp['Cabin_hash'] = df['Cabin'].map( lambda x: hash(x) % 10 )           # 利用 hash(x)函數取餘數，做特徵雜湊，並放到新的一欄
train_X = df_temp[:train_num]

estimator = LogisticRegression()
score = cross_val_score( estimator, train_X, train_Y, cv=5).mean()
print('score : ', score)
df_temp.head()

score :  0.7800075324838366


,Name,Sex,Ticket,Cabin,Embarked,Cabin_hash
0,155,1,720,185,3,5
1,286,0,816,106,0,2
2,523,0,914,185,3,5
3,422,0,65,70,3,7
4,22,1,649,185,3,5


In [141]:
# 'Cabin'計數編碼 + 'Cabin'特徵雜湊 + 邏輯斯迴歸
df_temp = pd.DataFrame()

for col in object_features:
    df_temp[col] = le.fit_transform(df[col])                        # 將其他特徵用 Label Encoding 處理 
df_temp['Cabin_hash'] = df['Cabin'].map(lambda x : hash(x) % 10 )   # 做特徵雜湊
df_temp['Cabin_count'] = df['Cabin_count']                          # 加入['Cabin_count']資料(計數編碼)
train_X = df_temp[:train_num]

estimator = LogisticRegression()
score = cross_val_score(estimator, train_X, train_Y, cv=5).mean()
print('score : ', score)
df_temp.head()

score :  0.7788776599083549


,Name,Sex,Ticket,Cabin,Embarked,Cabin_hash,Cabin_count
0,155,1,720,185,3,5,1014
1,286,0,816,106,0,2,2
2,523,0,914,185,3,5,1014
3,422,0,65,70,3,7,2
4,22,1,649,185,3,5,1014
